In [25]:
import cv2
import numpy as np

# Load an image
image = cv2.imread(r'C:\Users\kapitoo\Downloads\apple.jpg', 1)  # Read the image as grayscale

# Apply Canny edge detection
edges = cv2.Canny(image, 50, 150)  # You can adjust the threshold values

# Display the original and edges images
cv2.imshow('Original Image', image)
cv2.imshow('Canny Edges', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
import cv2
import numpy as np

def sobel_operator(image):
    # Sobel kernels
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    # Convolve the image with Sobel kernels
    gradient_x = convolve(image, sobel_x)
    gradient_y = convolve(image, sobel_y)

    # Compute gradient magnitude and direction
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    gradient_direction = np.arctan2(gradient_y, gradient_x)

    return gradient_magnitude, gradient_direction

def convolve(image, kernel):
    height, width = image.shape
    k_height, k_width = kernel.shape

    # Calculate the padding to handle the kernel size
    pad_height = k_height // 2
    pad_width = k_width // 2

    # Create an empty output image
    output_image = np.zeros((height, width))

    # Iterate over the input image and apply convolution
    for i in range(pad_height, height - pad_height):
        for j in range(pad_width, width - pad_width):
            neighborhood = image[i - pad_height:i + pad_height + 1, j - pad_width:j + pad_width + 1]
            output_image[i, j] = np.sum(neighborhood * kernel)

    return output_image

def canny_edge_detection(image, low_threshold, high_threshold):
    # Step 1: Apply Gaussian blur to the image
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

    # Step 2: Compute gradient using Sobel operators
    gradient_magnitude, gradient_direction = sobel_operator(blurred_image)

    # Step 3: Non-maximum suppression
    suppressed_image = non_maximum_suppression(gradient_magnitude, gradient_direction)

    # Step 4: Edge tracking by hysteresis
    edges = hysteresis_thresholding(suppressed_image, low_threshold, high_threshold)

    return edges.astype(np.uint8) * 255

def non_maximum_suppression(magnitude, direction):
    height, width = magnitude.shape
    suppressed = np.zeros_like(magnitude)

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            angle = direction[i, j]

            # Define gradient directions
            grad_dir = (0 if (angle < np.pi / 8) or (angle >= 7 * np.pi / 8) else
                        1 if (angle >= np.pi / 8) and (angle < 3 * np.pi / 8) else
                        2 if (angle >= 3 * np.pi / 8) and (angle < 5 * np.pi / 8) else
                        3)

            # Perform non-maximum suppression
            if grad_dir == 0:
                neighbors = (magnitude[i, j - 1], magnitude[i, j + 1])
            elif grad_dir == 1:
                neighbors = (magnitude[i - 1, j - 1], magnitude[i + 1, j + 1])
            elif grad_dir == 2:
                neighbors = (magnitude[i - 1, j], magnitude[i + 1, j])
            else:
                neighbors = (magnitude[i - 1, j + 1], magnitude[i + 1, j - 1])

            suppressed[i, j] = magnitude[i, j] if magnitude[i, j] >= max(neighbors) else 0

    return suppressed

def hysteresis_thresholding(image, low_threshold, high_threshold):
    strong_edges = (image >= high_threshold)
    weak_edges = (image >= low_threshold) & (image < high_threshold)

    # Perform edge tracking by hysteresis
    for i in range(1, image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            if weak_edges[i, j]:
                # Check if any of the 8 neighboring pixels is a strong edge
                if np.any(strong_edges[i-1:i+2, j-1:j+2]):
                    strong_edges[i, j] = True
                else:
                    weak_edges[i, j] = False

    return strong_edges.astype(np.uint8)

# Load an image
image = cv2.imread(r'C:\Users\kapitoo\Downloads\apple.jpg', cv2.IMREAD_COLOR)

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
canny_edges = canny_edge_detection(gray_image, 50, 150)

# Display the original and Canny edges images
cv2.imshow('Original Image', image)
cv2.imshow('Canny Edges', canny_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()
